In [0]:
import pyspark.sql.functions as F

In [0]:
df_source = spark.table("d_dnst_020_silver.silver_01.`01_sellout_consolidated`")

In [0]:
df_matched_internal = spark.table("d_dnst_020_silver.silver_03.`031_sellout_customers_int_matched`")

In [0]:
(
    df_matched_internal
    .filter(F.col("unique_id").isNull())
    .display()
)

In [0]:
df_joined = (
    df_matched_internal.alias("l")
    .join(
        df_matched_internal.alias("r"),
        on = F.expr("l.int_vat_number_match == r.unique_id and l.int_vat_number_match is not NULL"),
        how = "inner"
    )
    .selectExpr(
        "l.*",
        "r.customer_name as matched_name",
        "r.unique_id as matched_unique_id",
        # "r.dim_sellout_cust_id as matched_dim_sellout_cust_id",
        "r.customer_postal_code as matched_customer_postal_code",
        "r.vat_number as matched_vat_number",
        "r.customer_address as matched_customer_address",
        "r.customer_number as matched_customer_number",
        "r.dim_sellout_cust_id as matched_dim_sellout_cust_id"
    )
)

In [0]:
(
    df_joined
    .filter(F.col("customer_number").isin("KB09412", "KB10304"))
    .select("customer_name","unique_id", "vat_number", "int_vat_number_match")
    .display()
)

In [0]:
(
    df_joined
    .select("customer_number", "customer_name", "matched_name", "customer_address", "matched_customer_address", "customer_postal_code", "matched_customer_postal_code", "int_vat_number_match", "matched_unique_id")
    .filter(F.col("int_levenstein_name_address").isNull())
    .filter(F.col("matched_customer_postal_code") != F.col("customer_postal_code"))
    .display()
)